In [1]:
# importing all libraries needed
import numpy as np
import pandas as pd
import os
import time
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

In [2]:
#Stanting timming process
start = time.time()

In [3]:
# saving the main directory
main_directory = os.getcwd()

In [4]:
# subfolder_name = [f.name for f in os.scandir(main_directory) if f.is_dir()]
subfolder_with_path = ["" for i in range(4)]
subfolder_with_path[0] = main_directory + '/'+'01Activity_files'
subfolder_with_path[1] = main_directory + '/'+'02Volume_files'
subfolder_with_path[2] = main_directory + '/'+'03Volume_in_tissue_files'
subfolder_with_path[3] = main_directory + '/'+'zzz_input_and_reference_files'

In [5]:
###############################################################################################
# Importing information from "samples overview", "intensisties template" and "brain allen map"
###############################################################################################

# Changing directory to the "zzz_input_and_reference_files" folder
os.chdir(subfolder_with_path[-1])


# Reading "sample_overview.csv" file
samples_overview = pd.read_csv("sample_overview.csv", 
                               dtype={"SAMPLE": int,
                                      "hemisphere": "string","marker": "string",
                                      "TX_GROUP": "string"})

# Reading "gubra_intensities_template.csv" file
gubra_intensities_template = pd.read_csv("gubra_intensities_template.csv",
                                   dtype={"IDPath": "string", 
                                          "LabelID": int, 
                                          "raw": "string", 
                                          "LabelAbrv": "string"})
# Reading "brain_allen_map.csv" file
brain_gubra_map = pd.read_csv("brain_gubra_map_fixed.csv", 
                               dtype={"IDPath": "string", "LabelAbrv": "string",
                                      "raw": "string","allen_1": "string",
                                      "allen_2": "string","allen_3": "string",
                                      "allen_4": "string","allen_5": "string",
                                      "allen_6": "string","allen_7": "string",
                                      "fine": "string","medium": "string",
                                      "coarse": "string", "all":"string"})


In [6]:
#Saving drugs and markers' names from the "sample_overview" file

drugs = samples_overview['TX_GROUP'].unique()
markers = samples_overview['marker'].unique()

d = 0
m = 0
counters_names = []
while (m < len(markers)):
    while (d < len(drugs)):
        counters_names.append(drugs[d] + '_' + markers[m])
        d += 1
    m += 1
    d = 0

In [7]:
# Changing directory to the "02Volume_files" folder
os.chdir(subfolder_with_path[1])

#Counting number of files on "02Volume_files" folder
list = os.listdir(subfolder_with_path[1]) 
number_files = len(list)
number_volume_files = number_files

In [8]:
#Using and timing  a loop to process all files and populate the main data-frame

volume_data = [] #This array will contain df as elements

#Staring counters

i = 0 #General loop counter

#Zeroing the counters 
counters = np.zeros(len(counters_names))

#Creating counters array to contain sample's names as elements
counters_samples = [[] for j in range(len(counters_names))]

while i < number_files:

    ##############################################
    # Processing each file
    ##############################################
    
    file = list[i]
    # Saving sample's number
    filename_segmented = file.split('_')
    sample = filename_segmented[1]


    # Reading file's data
    header = ["PIXEL_COUNT"]
    pixels_count_volume = pd.read_csv(file, names = header)
    pixels_count_volume.insert(0, column = 'INTENSITY', value = range(0, len(pixels_count_volume))) 

    # Adding activity columns to intensitites template data-frame
    output = gubra_intensities_template.merge(pixels_count_volume, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    volume_label_name = 'VOLUME'
    output = output.rename(columns = {'PIXEL_COUNT':volume_label_name})
    output['VOLUME'] = output['VOLUME'].div(1000000).round(7)
    
    # Checking to which hemisphere the sample belongs to and erasing activity that belong to the other hemisphere
    hemisphere = samples_overview['hemisphere'].loc[samples_overview['SAMPLE'] == int(sample)]
    hemisphere = hemisphere.array
    if hemisphere[0] == 'left':
        output = output[output['LabelID'] > 20000]
    else:
        output = output[output['LabelID'] < 20000]

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
    
    # Merging activity and sample overview data-frames
    output= pd.merge(output,samples_overview[['SAMPLE',
                                             'hemisphere','marker',
                                             'TX_GROUP']],on='SAMPLE',how='inner')
    
    #Adding a column with the sample's nickname
    counters_index = counters_names.index(output['TX_GROUP'].iloc[0] + '_' + output['marker'].iloc[0])
    if output['SAMPLE'].iloc[0] not in counters_samples[counters_index]:
        counters[counters_index] += 1
        counters_samples[counters_index].append(output['SAMPLE'].iloc[0])
        nickname = 'v_' + output['TX_GROUP'].iloc[0] + '_' + str(int(counters[counters_index]))

    nickname_column = np.empty(len(output))    
    output['nickname'] = nickname_column
    output['nickname'] = nickname
    
    # Erasing the hemisphere label form the "raw" and the and "labelabrv" columns
    output["raw"] = output["raw"].str.replace('left_','')
    output["raw"] = output["raw"].str.replace('right_','')
    output["LabelAbrv"] = output["LabelAbrv"].str.replace('L','',1)
    output["LabelAbrv"] = output["LabelAbrv"].str.replace('R','',1)
    
    ##########################################################################################################################
    
    # store DataFrame in list
    volume_data.append(output)
    
    i += 1
    
#Joinning all elements of the array in a dataframe
volume_data = pd.concat(volume_data)

In [9]:
# Changing directory to the "03Volume_in_tissue_files" folder
os.chdir(subfolder_with_path[2])

#Counting number of files on "03Volume_in_tissue_files" folder
list = os.listdir(subfolder_with_path[2]) 
number_files = len(list)
number_volintissue_files = number_files

In [10]:
#Using and timing  a loop to process all files and populate the main data-frame

volume_intissue_data = [] #This array will contain df as elements

#Staring counters

i = 0 #General loop counter

#Zeroing the counters 
counters = np.zeros(len(counters_names))

#Creating counters array to contain sample's names as elements
counters_samples = [[] for j in range(len(counters_names))]

    ##############################################
    # Processing each file
    ##############################################

while i < number_files:


    file = list[i]
    # Saving sample's number 
    filename_segmented = file.split('_')
    sample = filename_segmented[1]

    # Reading file's data
    header = ["PIXEL_COUNT"]
    pixels_count_volume = pd.read_csv(file, names = header)
    pixels_count_volume.insert(0, column = 'INTENSITY', value = range(0, len(pixels_count_volume))) 

    # Adding activity columns to intensitites template data-frame
    output = gubra_intensities_template.merge(pixels_count_volume, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    volume_label_name = 'VOLUME_IN_TISSUE'
    output = output.rename(columns = {'PIXEL_COUNT':volume_label_name})
    output['VOLUME_IN_TISSUE'] = output['VOLUME_IN_TISSUE'].div(1000000).round(6)
    
    # Checking to which hemisphere the sample belongs to and erasing activity that belong to the other hemisphere
    hemisphere = samples_overview['hemisphere'].loc[samples_overview['SAMPLE'] == int(sample)]
    hemisphere = hemisphere.array
    if hemisphere[0] == 'left':
        output = output[output['LabelID'] > 20000]
    else:
        output = output[output['LabelID'] < 20000]

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
    
    # Merging activity and sample overview data-frames
    output= pd.merge(output,samples_overview[['SAMPLE',
                                             'hemisphere','marker',
                                             'TX_GROUP']],on='SAMPLE',how='inner')
    
    #Adding a column with the sample's nickname

    counters_index = counters_names.index(output['TX_GROUP'].iloc[0] + '_' + output['marker'].iloc[0])
    if output['SAMPLE'].iloc[0] not in counters_samples[counters_index]:
        counters[counters_index] += 1
        counters_samples[counters_index].append(output['SAMPLE'].iloc[0])
        nickname = 'vit_' + output['TX_GROUP'].iloc[0] + '_' + str(int(counters[counters_index]))

    nickname_column = np.empty(len(output))    
    output['nickname'] = nickname_column
    output['nickname'] = nickname
    
    # Erasing the hemisphere label form the "raw" and the and "labelabrv" columns
    output["raw"] = output["raw"].str.replace('left_','')
    output["raw"] = output["raw"].str.replace('right_','')
    output["LabelAbrv"] = output["LabelAbrv"].str.replace('L','',1)
    output["LabelAbrv"] = output["LabelAbrv"].str.replace('R','',1)
    
    ##########################################################################################################################
    
    # store DataFrame in list
    volume_intissue_data.append(output)
    
    i += 1
    
#Joinning all elements of the array in a dataframeo
volume_intissue_data = pd.concat(volume_intissue_data)

In [11]:
#calculating volume ratio
volumes_data = pd.merge(volume_intissue_data,volume_data[['raw','VOLUME','SAMPLE']],on=('raw','SAMPLE'),how='left')
volumes_data['volumes_ratio'] = volumes_data['VOLUME_IN_TISSUE'] / volumes_data['VOLUME'] 

In [12]:
#Pivoting dataframe to have nicknames as columns
pivoted_volumesintissue_data = pd.pivot_table(volume_intissue_data, values='VOLUME_IN_TISSUE', 
                                       index=['raw','SAMPLE'],columns=['nickname'], aggfunc=np.sum)
#Restarting index to activate'raw' and 'marker' as columns
pivoted_volumesintissue_data = pivoted_volumesintissue_data.reset_index()

In [13]:
#Pivoting dataframe to have nicknames as columns
pivoted_volumes_data = pd.pivot_table(volume_data, values='VOLUME', 
                                       index=['raw', 'SAMPLE'],columns=['nickname'], aggfunc=np.sum)
#Restarting index to activate'raw' and 'marker' as columns
pivoted_volumes_data = pivoted_volumes_data.reset_index()

In [14]:
# Changing directory to the "01Activity_files" folder
os.chdir(subfolder_with_path[0])

#Counting number of files on "01Activity_files" folder
list = os.listdir(subfolder_with_path[0]) 
number_files = len(list)
number_activity_files = number_files

In [15]:
#Using and timing  a loop to process all files and populate the main data-frame

activity_data = [] #This array will contain df as elements

#Staring counters

i = 0 #General loop counter

#Zeroing the counters 
counters = np.zeros(len(counters_names))

#Creating counters array to contain sample's names as elements
counters_samples = [[] for j in range(len(counters_names))]


    ##############################################
    # Innitiates loop to process each file
    ##############################################


while i < number_files:

    # Working with each file at a time
    file = list[i]
    
    # Saving sample's number
    filename_segmented = file.split('_')
    sample = filename_segmented[1]

    # Reading file's data
    regions_count_activity_fracc = pd.read_csv(file,low_memory=False)
    regions_count_activity_fracc = regions_count_activity_fracc[regions_count_activity_fracc.PIXEL_COUNT != "PIXEL_COUNT"]        

    # Adding-up regions' fracctions
    regions_fracc_1 = regions_count_activity_fracc[["INTENSITY_1", "INTENSITY_1_PERC"]]
    regions_fracc_2 = regions_count_activity_fracc[["INTENSITY_2", "INTENSITY_2_PERC"]]
    regions_fracc_3 = regions_count_activity_fracc[["INTENSITY_3", "INTENSITY_3_PERC"]]
    regions_fracc_1 = regions_fracc_1.rename(columns={"INTENSITY_1": "INTENSITY","INTENSITY_1_PERC" : "COUNTS"})
    regions_fracc_2 = regions_fracc_2.rename(columns={"INTENSITY_2": "INTENSITY","INTENSITY_2_PERC" : "COUNTS"})
    regions_fracc_3 = regions_fracc_3.rename(columns={"INTENSITY_3": "INTENSITY","INTENSITY_3_PERC" : "COUNTS"})
    total_region_activity = regions_fracc_1.append(regions_fracc_2, 
                                                   ignore_index=True).append(regions_fracc_3, ignore_index=True)
    total_region_activity["COUNTS"] = total_region_activity["COUNTS"].astype(float)
    total_region_activity = total_region_activity.groupby(['INTENSITY']).agg('sum').reset_index()
#     total_region_activity["COUNTS"] = total_region_activity["COUNTS"].round(0)
    total_region_activity["INTENSITY"] = total_region_activity["INTENSITY"].astype(int)
        
    # Adding activity columns to intensitites template data-frame
    output = gubra_intensities_template.merge(total_region_activity, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    activity_label_name = 'ACTIVITY'
    output = output.rename(columns = {'COUNTS':activity_label_name})
    
    # Checking to which hemisphere the sample belongs to and erasing activity that belong to the other hemisphere
    hemisphere = samples_overview['hemisphere'].loc[samples_overview['SAMPLE'] == int(sample)]
    hemisphere = hemisphere.array
    if hemisphere[0] == 'left':
        output = output[output['LabelID'] > 20000]
    else:
        output = output[output['LabelID'] < 20000]

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
    
    # Merging activity and sample overview data-frames
    output= pd.merge(output,samples_overview[['SAMPLE',
                                             'hemisphere','marker',
                                             'TX_GROUP']],on='SAMPLE',how='inner')
    
    
    #Adding a column with the sample's nickname

    counters_index = counters_names.index(output['TX_GROUP'].iloc[0] + '_' + output['marker'].iloc[0])
    if output['SAMPLE'].iloc[0] not in counters_samples[counters_index]:
        counters[counters_index] += 1
        counters_samples[counters_index].append(output['SAMPLE'].iloc[0])
        nickname = output['TX_GROUP'].iloc[0] + '_' + str(int(counters[counters_index]))

    nickname_column = np.empty(len(output))    
    output['nickname'] = nickname_column
    output['nickname'] = nickname

    # Erasing the hemisphere label form the "raw" and the and "labelabrv" columns
    output["raw"] = output["raw"].str.replace('left_','')
    output["raw"] = output["raw"].str.replace('right_','')
    output["LabelAbrv"] = output["LabelAbrv"].str.replace('L','',1)
    output["LabelAbrv"] = output["LabelAbrv"].str.replace('R','',1)
    
    ##########################################################################################################################
    
    # store DataFrame in list
    activity_data.append(output)
    
    i += 1
    
#Joinning all elements of the array in a dataframe
activity_data = pd.concat(activity_data)

In [16]:
#Replacing empty cells with zeros
activity_data['ACTIVITY'].fillna(0, inplace=True)

In [17]:
#Creating density dataframe
##Merging activity, volume and volume in tissue dataframes
density = pd.merge(activity_data,
                                  volumes_data[['raw','SAMPLE','VOLUME']],
                                  on=['raw','SAMPLE'],
                                  how='left')
density = pd.merge(density,
                           volume_intissue_data[['raw','SAMPLE','VOLUME_IN_TISSUE']],
                           on=['raw','SAMPLE'],
                           how='left')
#Creating the density nickname (d_nickname)
density['DENSITY'] = density['ACTIVITY'] / density['VOLUME_IN_TISSUE']

# Changing directory to the main path
os.chdir(main_directory)
#Printing all columns
density.to_csv('GUBRA_output_all_columns.csv', index=False)

density['d_nickname'] = 'd_' + density['nickname']

In [18]:
#Pivoting dataframe to have d_nicknames as columns
pivoted_density = pd.pivot_table(density, values='DENSITY', 
                                       index=['raw', 'SAMPLE'],columns=['d_nickname'], aggfunc=np.sum)
#Restarting index to activate'raw' and 'marker' as columns
pivoted_density = pivoted_density.reset_index()

In [19]:
#Merging activity and volume ratio to filter activity where the volume ratio is below 50%
filtered_activity_data = pd.merge(activity_data,
                                  volumes_data[['raw','SAMPLE','volumes_ratio']],
                                  on=['raw','SAMPLE'],
                                  how='left')
filtered_activity_data.loc[(filtered_activity_data['volumes_ratio'] < 0.5),'ACTIVITY'] = ''

In [20]:
#Pivoting dataframe to have nicknames as columns
pivoted_activity_data = pd.pivot_table(filtered_activity_data, values='ACTIVITY', 
                               index=['raw','hemisphere','marker','SAMPLE'],columns=['nickname'], aggfunc=np.sum)

#Restarting index to activate'hemisphere','marker'and 'rater' as columns
pivoted_activity_data = pivoted_activity_data.reset_index()

In [21]:
#combining all dataframes
gubra_and_activity = pd.merge(brain_gubra_map,pivoted_activity_data,on='raw',how='right')
cell_count_output = pd.merge(gubra_and_activity,pivoted_volumesintissue_data,on=['raw','SAMPLE'],how='left')
cell_count_output = pd.merge(cell_count_output,pivoted_volumes_data,on=['raw','SAMPLE'],how='left')
cell_count_output = pd.merge(cell_count_output,pivoted_density,on=['raw','SAMPLE'],how='left')

In [22]:
# Changing directory to the main path
os.chdir(main_directory)

In [23]:
#creating the "cell_count_output.csv" file
cell_count_output.to_csv('GUBRA_output_for_ADX.csv', index=False)

In [25]:
########Creating output for Dan's Excel analysis


#Chaging SAMPLE elements to integer and then to string when adding the letter "S"
filtered_activity_data['SAMPLE_STR'] = filtered_activity_data['SAMPLE'].astype(int)
filtered_activity_data['SAMPLE_STR'] = filtered_activity_data['SAMPLE_STR'].astype(str)
filtered_activity_data['SAMPLE_STR'] = 'S' + filtered_activity_data['SAMPLE_STR']

#Pivoting dataframe to have sample as columns
pivoted_activity_Dan = pd.pivot_table(filtered_activity_data, values='ACTIVITY', 
                               index=['raw','hemisphere',
                                      'marker','SAMPLE'],
                                      columns=['SAMPLE_STR','TX_GROUP'], aggfunc=np.sum)

#Restarting index to activate'hemisphere','marker' as columns
pivoted_activity_Dan = pivoted_activity_Dan.reset_index()

#Adding GUBRA map columns
gubra_and_activity_Dan = pd.merge(brain_gubra_map,pivoted_activity_Dan,on='raw',how='right')

#Editing output
gubra_and_activity_Dan.rename(columns={('SAMPLE', ''): 'SAMPLE',
                                       ('hemisphere', ''): 'hemisphere',
                                       ('marker', ''): 'marker',
                                       'raw': 'LabelName'}, inplace=True)
gubra_and_activity_Dan = gubra_and_activity_Dan.drop(columns=['allen_1', 'allen_2',
                                     'allen_3', 'allen_4',
                                     'allen_5', 'allen_6',
                                     'allen_7', 'fine',
                                     'medium', 'coarse','all',
                                    'SAMPLE', 'hemisphere','marker'])
gubra_and_activity_Dan = gubra_and_activity_Dan.groupby('IDPath').apply(lambda x: x\
                                                                        .fillna(method='ffill')\
                                                                        .fillna(method='bfill')\
                                                                        .drop_duplicates()\
                                                                        .reset_index(drop=True))

# gubra_and_activity_Dan.columns = gubra_and_activity_Dan.columns.values.str.replace('[),(]', '')
# gubra_and_activity_Dan.columns = gubra_and_activity_Dan.columns.values.str.replace('[,]', '_')



In [26]:
#editing columns names from ('sample43', 'psilocybin') to S43_psilocybin
columns_name_array = gubra_and_activity_Dan.columns.values
#dividing into two arrays, one with str and one with tuple
#"cna" = column_name+array
number_of_cols = columns_name_array.size
cna_1 = columns_name_array[0:3]
cna_2 = columns_name_array[-(number_of_cols - 3):]
str = ''
for item in cna_2:
    str = str +'_'.join(item) + ' ' 
str = str.split(' ')[:-1]
new_columns_name = np.concatenate([cna_1, str]) 
new_columns_name
#Asingning new column names to the dataframe
gubra_and_activity_Dan = gubra_and_activity_Dan.set_axis(new_columns_name, axis=1, inplace=False)

In [42]:
#Ordering columns so all saline samples are fisrt
cols = gubra_and_activity_Dan.columns.tolist()
cols_first_3 = cna_1
cols_samples = cols[-(number_of_cols - 3):]

def w_len(e):
    len_e = len(e)
    if e.endswith("saline"):
        len_e = 1
    return len_e

cols_samples.sort(key=w_len)
new_cols = np.concatenate((cols_first_3, cols_samples), axis=None)

gubra_and_activity_Dan = gubra_and_activity_Dan[new_cols]

gubra_and_activity_Dan

,,IDPath,LabelAbrv,LabelName,S51_saline,S53_saline,S43_psilocybin,S44_psilocybin,S46_psilocybin,S47_psilocybin,S48_psilocybin,S50_psilocybin,S52_psilocybin,S54_psilocybin,S56_psilocybin
IDPath,,,,,,,,,,,,,,,
/997/,0,/997/,root,root,12184.8,11185.1,6817.086,3654.76,3477.64,11518.2,4812.09,3892.38,7887.16,10952.2,5444.07
/997/1009/,0,/997/1009/,fiber tracts,fiber_tracts,1648.01,1803.22,1362.769,1751.84,1856.2,1532.07,1582.3,1531.87,1670.74,2646.26,1678.86
/997/1009/1000/760/102/,0,/997/1009/1000/760/102/,nst,nigrostriatal_tract,100.341,154.205,43.200,109.315,91.148,78.148,74.145,43.502,166.327,131.205,41.576
/997/1009/1000/863/,0,/997/1009/1000/863/,rust,rubrospinal_tract,810.646,791.676,374.805,344.827,437.666,393.167,346.535,240.884,486.192,693.927,360.589
/997/1009/1000/863/397/,0,/997/1009/1000/863/397/,vtd,ventral_tegmental_decussation,44.677,19.064,0.000,5.333,16.615,59.729,9.5,5,21.274,53.814,20.972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/997/8/567/688/703/319/334/,0,/997/8/567/688/703/319/334/,BMAp,Basomedial_amygdalar_nucleus_posterior_part,979.259,1350.21,813.844,1094.75,1845.12,686.723,1148.86,1426.21,1819.3,1833.87,1630.54
/997/8/567/688/703/583/,0,/997/8/567/688/703/583/,CLA,Claustrum,625.643,1005.87,774.092,1201.64,1773.95,739.655,1627.73,1610.8,1623.16,1542.62,1626.75
/997/8/567/688/703/780/,0,/997/8/567/688/703/780/,PA,Posterior_amygdalar_nucleus,1621.59,2133.12,945.919,889.157,1453.95,630.717,851.603,1106.03,2245.92,2503.68,2029.25


In [43]:
gubra_and_activity_Dan.to_csv('GUBRA_act_output_Dan_Excel.csv', index=False)

In [44]:
#creating the module to ask to the user if they want to print and additional data frame with selected columns
# all_columns = density.columns.values
# possible_columns = np.delete(all_columns, np.where(all_columns == 'raw'))
# possible_columns = np.delete(possible_columns, np.where(possible_columns == 'd_nickname'))

In [45]:
#Ending timming process
end = time.time()

# Printing how many files were processed and how much time the process took
files_processed = number_activity_files = number_files+ number_volume_files + number_volintissue_files
print(files_processed,' Files processed ')
print(' ')
print(' Execution time:', round((end - start),2), 'seconds') 

33  Files processed 
 
 Execution time: 74404.84 seconds
